[Tensorboard pytorch tutorial](https://krishansubudhi.github.io/deeplearning/2020/03/24/tensorboard-pytorch.html)

Clear everything first 

In [19]:
! powershell "echo 'checking for existing tensorboard processes'"
! powershell "ps | Where-Object {$_.ProcessName -eq 'tensorboard'}"

! powershell "ps | Where-Object {$_.ProcessName -eq 'tensorboard'}| %{kill $_}"

! powershell "echo 'cleaning tensorboard temp dir'"
! powershell "rm -Force -Recurse $env:TEMP\.tensorboard-info\*" 

! powershell "ps | Where-Object {$_.ProcessName -eq 'tensorboard'}"
! powershell "rm -Force -Recurse runs\*"

checking for existing tensorboard processes
cleaning tensorboard temp dir


# Create Summary writer

In [20]:
from torch.utils.tensorboard import SummaryWriter
# Writer will output to ./runs/ directory by default
writer = SummaryWriter('runs/tensorboard_bert')

# Load bert model

In [21]:
import transformers, torch
from transformers import BertForMaskedLM, BertTokenizer
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [22]:
#text = "I am using tensorboard in pytorch. I am trying tensorboard to visualize BERT model."
def get_tensors(text):
    tokens = tokenizer.tokenize(text)
    tokens.insert(0, tokenizer.cls_token)
    tokens.append(tokenizer.sep_token)

    correct_tensors = torch.LongTensor(tokenizer.convert_tokens_to_ids(tokens)).unsqueeze(0)

    import numpy as np
    for _ in range(len(tokens)//5):
        i = np.random.randint(1, len(tokens)-1)
        tokens[i] = tokenizer.mask_token


    tensors = torch.LongTensor(tokenizer.convert_tokens_to_ids(tokens)).unsqueeze(0)
    #print(tokens)
    return  tokens, tensors, correct_tensors
#get_tensors(text)

# Adding model graph and embeddings 

In [23]:
writer.add_graph(model, torch.ones(1,128, dtype = torch.long))

# Adding scalars and histograms

In [29]:
def log_grads(step,model):
    for name,param in model.named_parameters():
        if ('bert.encoder.layer' in name and 'bert.encoder.layer.12' not in name):
            continue
        flat_params =  param.view(-1).data.numpy()
        writer.add_histogram('weight/'+name, flat_params, step)
        if param.view(-1).grad:
            flat_params =  param.view(-1).grad.numpy()
            writer.add_histogram('grads/'+name, flat_params, step)
        
#log_grads(0,model)

In [33]:
from torch.optim import adam
optimizer = adam.Adam( model.parameters(),0.1)
def run_step(step, model, tensors, correct_tensors):
    
    #forward
    loss,prediction_scores =  model.forward(tensors, masked_lm_labels = correct_tensors)
    
    #scalars
    preds = torch.argmax(prediction_scores, dim = -1)
    acc = torch.sum(correct_tensors[:, 1:-1]==preds[:,1:-1]).item()/correct_tensors.size()[-1]
    writer.add_scalar('Loss', loss.item(), step)
    writer.add_scalar('Accuracy', acc, step)
    
    
    #backwars
    loss.backward()
    log_grads(step, model)
    
    optimizer.step()
    optimizer.zero_grad
    
    return preds

In [34]:
texts = ["I am using tensorboard in pytorch. I am trying tensorboard to visualize BERT model.",
       "This is not scaling.",
       "Working from home these days because of lockdown.",
       "Hidden cup 3 for age of empires is awesome"]
for i,text in enumerate(texts):
    print (text)
    tokens, tensors, correct_tensors = get_tensors(text)
    preds =  run_step(i, model, tensors, correct_tensors)
    predicted_tokens = tokenizer.convert_ids_to_tokens(preds.squeeze().numpy())
    print(tokens,'\n')
    print(predicted_tokens)
    print()

I am using tensorboard in pytorch. I am trying tensorboard to visualize BERT model.
['[CLS]', 'i', 'am', '[MASK]', 'tensor', '##board', '[MASK]', '[MASK]', '##yt', '##or', '##ch', '.', 'i', 'am', '[MASK]', 'tensor', '##board', 'to', 'visual', '##ize', 'bert', 'model', '.', '[SEP]'] 

['.', 'i', 'am', 'the', 'tensor', '##board', 'for', 'w', '##yt', '##or', '##ch', '.', 'i', 'am', 'a', 'tensor', '##board', 'to', 'visual', '##ize', 'the', 'model', '.', '.']

This is not scaling.
['[CLS]', 'this', 'is', 'not', 'scaling', '[MASK]', '[SEP]'] 

['?', '?', '?', '?', '?', '?', '?']

Working from home these days because of lockdown.
['[CLS]', 'working', 'from', 'home', 'these', 'days', '[MASK]', 'of', 'lock', '[MASK]', '.', '[SEP]'] 

['awesome', 'awesome', 'awesome', 'awesome', 'awesome', 'awesome', 'awesome', 'awesome', 'awesome', 'awesome', 'awesome', 'awesome']

Hidden cup 3 for age of empires is awesome
['[CLS]', '[MASK]', 'cup', '3', 'for', 'age', '[MASK]', 'empires', 'is', 'awesome', '[SE

In [35]:
writer.flush()

In [28]:
writer.close()

In [ ]:
# Run in a new anaconda powershell

# ! pwd
# ! dir runs
# ! tensorboard --logdir="C:\Users\krkusuk\projects\tensorboard\runs"